## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-27-00-53-12 +0000,nypost,Chilling discovery made days after mom and her...,https://nypost.com/2025/12/26/us-news/chilling...
1,2025-12-27-00-50-38 +0000,nyt,Trump’s Claims About Nigeria Strike Belie a Co...,https://www.nytimes.com/2025/12/26/world/afric...
2,2025-12-27-00-47-52 +0000,nyt,Driver Livestreaming on TikTok Is Charged Afte...,https://www.nytimes.com/2025/12/26/us/tea-tyme...
3,2025-12-27-00-46-44 +0000,nyt,Police Respond to Report of a Shooting at an I...,https://www.nytimes.com/2025/12/26/us/active-s...
4,2025-12-27-00-38-04 +0000,nyt,What Went Wrong Before Hong Kong’s Inferno,https://www.nytimes.com/2025/12/26/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2395/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,25
6,year,13
551,christmas,11
98,killed,9
144,new,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
6,2025-12-27-00-33-08 +0000,nyt,Zelensky Says He’ll Meet With Trump Soon to Di...,https://www.nytimes.com/2025/12/26/world/europ...,66
39,2025-12-26-21-22-50 +0000,nypost,Trump tells Post there’s a ‘good shot’ at reac...,https://nypost.com/2025/12/26/us-news/trump-te...,66
107,2025-12-26-16-05-24 +0000,wapo,"Zelensky says he will meet Trump soon, citing ...",https://www.washingtonpost.com/world/2025/12/2...,63
110,2025-12-26-15-47-53 +0000,cbc,"Zelenskyy, Trump to meet in Florida on Sunday ...",https://www.cbc.ca/news/world/zelenskyy-trump-...,62
94,2025-12-26-17-29-14 +0000,nypost,Trump on Nigeria strikes: ISIS ‘got a very bad...,https://nypost.com/2025/12/26/us-news/trump-sp...,57


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
6,66,2025-12-27-00-33-08 +0000,nyt,Zelensky Says He’ll Meet With Trump Soon to Di...,https://www.nytimes.com/2025/12/26/world/europ...
116,48,2025-12-26-15-24-08 +0000,nypost,Dozens of Islamic State suspects plotting Chri...,https://nypost.com/2025/12/26/world-news/dozen...
76,33,2025-12-26-19-13-16 +0000,nypost,Heartbreaking note reveals troubled past of dr...,https://nypost.com/2025/12/26/us-news/heartbre...
33,26,2025-12-26-21-48-11 +0000,nypost,NYC commuters face OMNY problems just days bef...,https://nypost.com/2025/12/26/us-news/commuter...
51,26,2025-12-26-20-25-56 +0000,nypost,Minnesota Lt. Gov. Peggy Flanagan wears hijab ...,https://nypost.com/2025/12/26/us-news/minnesot...
8,25,2025-12-26-23-58-34 +0000,nypost,Hochul hasn’t served up Trump’s ‘no tax on tip...,https://nypost.com/2025/12/26/us-news/hochul-w...
11,23,2025-12-26-23-49-29 +0000,bbc,Two killed in suspected Palestinian ramming an...,https://www.bbc.com/news/articles/c9d9znp22pyo...
151,23,2025-12-26-10-30-00 +0000,wsj,Tens of thousands of imports are being stopped...,https://www.wsj.com/economy/trade/customs-crac...
120,23,2025-12-26-15-01-16 +0000,bbc,Blast at Syrian mosque during Friday prayers l...,https://www.bbc.com/news/articles/cx25387dj8no...
9,22,2025-12-26-23-58-16 +0000,nypost,LA Mayor Karen Bass calls it ‘sad’ that Hispan...,https://nypost.com/2025/12/26/us-news/la-mayor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
